In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import torch
from scipy.stats import norm
import time
torch.set_default_dtype(torch.float64)

# Preliminaries 0: Monte Carlo function with confidence interval (from M. Lemaire and adapted for pytorch)

In [ ]:
def monte_carlo(sample, proba = 0.95):
   
    mean = torch.mean(sample)
    var = torch.var(sample)#, ddof=1)
    alpha = 1 - proba 
    quantile = stats.norm.ppf(1 - alpha/2)  # fonction quantile 
    s = list(sample.size())
    ci_size = quantile * torch.sqrt(var / s[0])
    result = { 'mean': mean, 'var': var, 
               'lower': mean - ci_size, 
               'upper': mean + ci_size }
    return result

# Preliminaries 1: Black & Scholes true values for a European Call and its Delta and Gamma

In [ ]:
def CallPrice(spot, rfr, vol, strike, T):
    
    d1 = (np.log(spot/strike) + (rfr + vol**2 /2) * T)/(vol * np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    
    return spot * norm.cdf(d1) - strike * np.exp(-rfr * T) * norm.cdf(d2)

In [ ]:
def DeltaCall(spot, rfr, vol, strike, T):
    
    d1 = (np.log(spot/strike) + (rfr + vol**2 /2) * T)/(vol * np.sqrt(T))
    return norm.cdf(d1)

In [ ]:
def GammaCall(spot, rfr, vol, strike, T):
    
    d1 = (np.log(spot/strike) + (rfr + vol**2 /2) * T)/(vol * np.sqrt(T))
    return norm.pdf(d1)/(spot * vol * np.sqrt(T))

# Preliminaries 2: Payoff Classes

In [ ]:
class CallPayoff:
    
    def __init__(self, strike):
        
        self._strike = torch.tensor(strike)
        
    
    def ComputePayoff(self, x):
        
        return torch.maximum(x - self._strike, torch.tensor(0))

In [ ]:
class PutPayoff:
    
    def __init__(self, strike):
        
        self._strike = torch.tensor(strike)
        
    
    def ComputePayoff(self, x):
        
        return torch.maximum(self._strike - x, torch.tensor(0))

In [ ]:
class BasketCallPayoff:
    
    def __init__(self, weights, strike):
        
        self._strike =torch.tensor(strike)
        self._weights = torch.reshape(weights, shape=(weights.shape[0], 1))
        
    
    def ComputePayoff(self, x):
        
        tempVar = x @ self._weights
        return torch.maximum(tempVar - self._strike, torch.tensor(0))

In [ ]:
class BasketPutPayoff:
    
    def __init__(self, weights, strike):
        
        self._strike = torch.tensor(strike)
        self._weights = torch.reshape(weights, shape=(weights.shape[0], 1))
        
    
    def ComputePayoff(self, x):
        
        tempVar = x @ self._weights
        return torch.maximum(self._strike - tempVar, torch.tensor(0))

# I- Sensitivities computation with finite difference in a Black & Scholes model for European and American options

La classe ci-dessous permet de calculer les sensitivités d'un call ou put européen ou américain dans un modèle de Black & Scholes par différences finies

In [ ]:
class FD1D:
    
    def __init__(self, nbSteps, nbSimus, payoff, m=5):
        
        self._payoff = payoff
        self._nbSteps = nbSteps
        self._nbSimus = nbSimus
        self._payoffs = torch.empty(size=(nbSimus,mZ))
        #self._path = torch.empty(size=(nbSimus,mZ))
        self._gaussians = np.random.normal(size=(nbSimus, 1, nbSteps))
        self._gaussians = torch.tensor(self._gaussians)
        self._lastGaussians = np.random.normal(size=(self._nbSimus, 1))
        self._lastGaussians = torch.tensor(self._lastGaussians)
        self._m = m
        
    
    def BSPath(self, startTime, endTime, spot, vol, rfr):
        
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
        
        #Reinitializing the paths in case they were already simulated, and initializing the original time step
        path = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        path[:, 0, 0] = spot # paths initialization to the initial spots
        
        for i in range(1, self._nbSteps):
            
            p = path[:, 0, i-1].clone()
            path[:, 0, i] = p + rfr * dt * p + \
                                vol * p * torch.sqrt(dt) * self._gaussians[:, 0, i]
            
        return path
    
    
    def MCPrice(self, startTime, endTime, spot, vol, rfr):
        
        start = time.time()
        T = endTime - startTime
        T = torch.tensor(T)
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
        
        path = self.BSPath(startTime, endTime, spot, vol, rfr)
        
        #Reinitializing the values if we reuse the function for the same vibrato object
        self._payoffs = torch.empty(size=(nbSimus,mZ))
        
        lastSpots = torch.empty(size=(self._nbSimus, 1, 2)) # We have to generate the standard value, and the antithetic counterpart
        payoffs = torch.empty(size=(self._nbSimus, 1))
        payoffsAnti = torch.empty(size=(self._nbSimus, 1))
        
        lastSpots[:, 0, 0] = path[:, 0, -1] * (1 + rfr * dt + vol * torch.sqrt(dt) * self._lastGaussians[:, 0])
        lastSpots[:, 0, 1] = path[:, 0, -1] * (1 + rfr * dt - vol * torch.sqrt(dt) * self._lastGaussians[:, 0])

        payoffs[:, 0] = self._payoff.ComputePayoff(lastSpots[:, 0, 0])
        payoffsAnti[:, 0] = self._payoff.ComputePayoff(lastSpots[:, 0, 1])
        
        self._payoffs = 0.5 * (payoffs + payoffsAnti)
        end=time.time()
        print(end-start)
        return torch.exp(-rfr * T) * payoffs  # 0.5 * torch.exp(-rfr * T) * (payoffs + payoffsAnti)
    
    
    def GetRegressorMatrix(self, spots):
        """
        Function to get the regressor matrix using a polynomial basis.
        """
        regressorMat = torch.empty(size=(self._nbSimus, self._m+1))
        
        for i in range(self._m+1):
            regressorMat[:, i] = torch.pow(spots, i)
                
        return regressorMat
    
    
    def MCPriceLS(self, startTime, endTime, spot, vol, rfr):
        
        start=time.time()
        
        T = endTime - startTime
        T = torch.tensor(T)
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
        
        path = self.BSPath(startTime, endTime, spot, vol, rfr)
        
        #Reinitializing the values if we reuse the function for the same vibrato object
        self._payoffs = torch.empty(size=(nbSimus,mZ))
        
        lastSpots = torch.empty(size=(self._nbSimus, 1)) # We have to generate the standard value, and the antithetic counterpart
        payoffs = torch.empty(size=(self._nbSimus, 1))
        payoffsAnti = torch.empty(size=(self._nbSimus, 1))
        
        lastSpots[:, 0] = path[:, 0, -1] * (1 + rfr * dt + vol * torch.sqrt(dt) * self._lastGaussians[:, 0])

        payoffs[:, 0] = self._payoff.ComputePayoff(lastSpots[:, 0])
        
        optimalPayoffs = torch.exp(-rfr * T) * payoffs
        

        for i in range(1, self._nbSteps):
            
            
            #Récupération des spots pour cette time step
            spots = path[:, 0, -(i)]
            #Calcul des fonctions de "continuation"
            matX = self.GetRegressorMatrix(spots)
            theta = torch.linalg.inv(matX.T @ matX) @ matX.T @ optimalPayoffs
            theta = torch.tensor(theta)
            valueApprox = matX @ theta
            
            payoffs = torch.exp(-rfr * T * (self._nbSteps - i)/self._nbSteps) * self._payoff.ComputePayoff(spots)
            
            for j in range(self._nbSimus):
                if payoffs[j] >= valueApprox[j]:
                    optimalPayoffs[j] = payoffs[j]
    
        initialPayoff = torch.exp(-rfr * T) * self._payoff.ComputePayoff(spot)
        
        end=time.time()
        print(end-start)
        
        return torch.maximum(initialPayoff, optimalPayoffs)#.mean())
        
    
    def ComputeDelta(self, startTime, endTime, spot, vol, rfr, eps):
        
        start=time.time()
        
        a = self.MCPrice(startTime, endTime, spot+ eps, vol, rfr)
        b = self.MCPrice(startTime, endTime, spot-eps, vol, rfr)
        
        delta = (a-b)/(2*eps)
        end=time.time()
        print(end-start)
        return delta
    
    
    def ComputeVega(self, startTime, endTime, spot, vol, rfr, eps):
        
        vega = (self.MCPrice(startTime, endTime, spot, vol+eps, rfr) - 
                self.MCPrice(startTime, endTime, spot, vol-eps, rfr))/(2*eps)
        return vega
    
    
    def ComputeGamma(self, startTime, endTime, spot, vol, rfr, eps):
        
        start=time.time()
        gamma = (self.MCPrice(startTime, endTime, spot+eps, vol, rfr) + self.MCPrice(startTime, endTime, spot-eps, vol, rfr) 
               - 2 * self.MCPrice(startTime, endTime, spot, vol, rfr))/eps**2
        end=time.time()
        print(end-start)
        return gamma
    
    def ComputeVegaVega(self, startTime, endTime, spot, vol, rfr, eps):
        
        volEps = vol + eps
        volEpsM = vol - eps
        vegaVega = (self.MCPrice(startTime, endTime, spot, volEps, rfr) + self.MCPrice(startTime, endTime, spot, volEpsM, rfr) 
                - 2 * self.MCPrice(startTime, endTime, spot, vol, rfr))/eps**2
        return vegaVega
    
    
    def ComputeDeltaAmerican(self, startTime, endTime, spot, vol, rfr, eps):
        
        a = self.MCPriceLS(startTime, endTime, spot+ eps, vol, rfr)
        b = self.MCPriceLS(startTime, endTime, spot-eps, vol, rfr)
        print(a)
        print(b)
        delta = (a-b)/(2*eps)
        return delta
    
    
    def ComputeGammaAmerican(self, startTime, endTime, spot, vol, rfr, eps):
        
        gamma = (self.MCPriceLS(startTime, endTime, spot+eps, vol, rfr) + self.MCPriceLS(startTime, endTime, spot-eps, vol, rfr) 
               - 2 * self.MCPriceLS(startTime, endTime, spot, vol, rfr))/eps**2
        return gamma

In [ ]:
class FD1DHeston:
    
    def __init__(self, nbSteps, nbSimus, payoff):
        
        self._payoff = payoff
        self._nbSteps = nbSteps
        self._nbSimus = nbSimus
        self._payoffs = torch.empty(size=(nbSimus,mZ))
        self._gaussians = np.random.normal(size=(nbSimus, 2, nbSteps))
        self._gaussians = torch.tensor(self._gaussians.astype(np.float32))
        self._lastGaussians = np.random.normal(size=(self._nbSimus, 2))
        self._lastGaussians = torch.tensor(self._lastGaussians)
        
    
    def HestonPath(self, startTime, endTime, spot, vol, rfr, correl, kappa, eta, xi):
        
        
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
       # W = Brownian1D(self._nbSteps, self._nbSimus)
       # W.Simulate(startTime, endTime)
       # self._gaussians = W._gaussians
        
        #Reinitializing the paths in case they were already simulated, and initializing the original time step
        path = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        path[:, 0, 0] = spot # paths initialization to the initial spots
        
        #Reinitializing the paths in case they were already simulated, and initializing the original time step
        pathVariance = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        pathVariance[:, 0, 0] = vol # paths initialization to the initial variance
        
        matGauss = np.array([[1, 0], [correl, torch.sqrt(1 - torch.square(torch.tensor(correl)))]])
        matGauss = torch.tensor(matGauss.astype(np.float32))
        
        
        for i in range(1, self._nbSteps):
            
            p = path[:, 0, i-1].clone()
            pV = pathVariance[:, 0, i-1].clone()
            
            gaussianCorr = matGauss @ torch.reshape(self._gaussians[:, :, i], shape=(self._nbSimus, 2, 1))
            
            path[:, 0, i] = p + rfr * dt * p + torch.sqrt(pV) * p * torch.sqrt(dt) * gaussianCorr[:, 0, 0]
            
            pathVariance[:, 0, i] = pV + kappa * (eta - pV) * dt + \
                                            xi * torch.sqrt(pV * dt) *  gaussianCorr[:, 1, 0]
            
            
        return path, pathVariance
    
    
    def MCPrice(self, startTime, endTime, spot, vol, rfr, correl, kappa, eta, xi):
        
        
        start = time.time()
        
        T = endTime - startTime
        T = torch.tensor(T)
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
        
        
        spot=torch.tensor(spot)
        vol=torch.tensor(vol)
        rfr=torch.tensor(rfr)
        correl=torch.tensor(correl)
        kappa=torch.tensor(kappa)
        eta=torch.tensor(eta)
        xi=torch.tensor(xi)
        
        path, pathVariance = self.HestonPath(startTime, endTime, spot, vol, rfr, correl, kappa, eta, xi)
        
        #Reinitializing the values if we reuse the function for the same vibrato object
        self._payoffs = torch.empty(size=(nbSimus,1))
        
        lastSpots = torch.empty(size=(self._nbSimus, 1, 2)) # We have to generate the standard value, and the antithetic counterpart
        payoffs = torch.empty(size=(self._nbSimus, 1))
        payoffsAnti = torch.empty(size=(self._nbSimus, 1))
        
        
        lastSpots[:, 0, 0] = path[:, 0, -1] * (1 + rfr * dt + torch.sqrt(pathVariance[:, 0, -1] * dt) * self._lastGaussians[:, 0])
        lastSpots[:, 0, 1] = path[:, 0, -1] * (1 + rfr * dt - torch.sqrt(pathVariance[:, 0, -1] * dt) * self._lastGaussians[:, 0])
        
        payoffs[:, 0] = self._payoff.ComputePayoff(lastSpots[:, 0, 0])
        payoffsAnti[:, 0] = self._payoff.ComputePayoff(lastSpots[:, 0, 1])
        
        end=time.time()
        print(end-start)
        return 0.5 * torch.exp(-rfr * T) * (payoffs + payoffsAnti)
    
    
    def ComputeDelta(self, startTime, endTime, spot, vol, rfr, correl, kappa, eta, xi, eps):
        
        start=time.time()
        
        a = self.MCPrice(startTime, endTime, spot+ eps, vol, rfr, correl, kappa, eta, xi)
        b = self.MCPrice(startTime, endTime, spot-eps, vol, rfr, correl, kappa, eta, xi)
        
        delta = (a-b)/(2*eps)
        end=time.time()
        print(end-start)
        return delta
    
    
    def ComputeGamma(self, startTime, endTime, spot, vol, rfr, correl, kappa, eta, xi, eps):
        
        start=time.time()
        gamma = (self.MCPrice(startTime, endTime, spot+eps, vol, rfr, correl, kappa, eta, xi) + 
                 self.MCPrice(startTime, endTime, spot-eps, vol, rfr, correl, kappa, eta, xi) 
               - 2 * self.MCPrice(startTime, endTime, spot, vol, rfr, correl, kappa, eta, xi))/eps**2
        end=time.time()
        print(end-start)
        return gamma

# II - Case of a European Call with Black & Scholes model

## Black-Scholes path generation

This class generates uni-dimensional Black & Scholes paths for the underlying and the tangent process until the time step $N-1$ (following the paper). The paths are generated using:

$X_{k+1} = X_k + rhX_k + X_k \sigma \sqrt{h}Z_{k+1}$

$Y_{k+1} = Y_k + rhY_k + \partial_{\theta}(rh)X_k + \left(Y_k \sigma \sqrt{h} + \partial_{\theta}(\sigma \sqrt{h})X_k\right)Z_{k+1}$

with $Y_0 = \partial_{\theta}X_0$, i.e. $Y_0 = 1$ if $\theta = x$, the initial value of the underlying, and $Y_0 = 0$ else.

In order to apply the vibrato differentiation to American option, we also keep in memory the terms $\mu_k = X_k + rhX_k$.

Pour Vibrato à l'ordre deux, on a les processus tangents (avec $Y_t^{ij}:=\frac{d^2}{d\theta_i d\theta_j}X_t(\theta_i, \theta_j)$):

$Y_t^{xx} = rY_t^{xx}dt + \sigma Y_t^{xx}dW_t$, $Y_0^{xx} = 0$

$Y_t^{\sigma \sigma} = rY_t^{\sigma \sigma}dt + (2Y_t^{\sigma} + \sigma Y_t^{\sigma \sigma})dW_t$, $Y_0^{\sigma \sigma} = 0$

$Y_t^{\sigma x} = rY_t^{\sigma x} + (Y_t^{x} + \sigma Y_t^{\sigma x})dW_t$, $Y_0^{\sigma x}=0$

On voit notamment que le processus $Y^{xx}$ est constant égal à zéro.

In [ ]:
class BlackScholes1D:
    
    def __init__(self, spot, rfr, vol, nbSteps, nbSimus):
        """
        parameters:
        - spot(array): initial value of the underlying
        - rfr(double): risk free rate
        - vol(array): Black & Scholes volatility of the underlying
        - nbSteps(int): number of discretization steps
        - nbSimus(int): nb of Monte Carlo simulations
        """
        self._spot = torch.tensor(spot, requires_grad=True)  
        self._rfr = torch.tensor(rfr, requires_grad=True)
        self._vol = torch.tensor(vol, requires_grad=True)
        self._nbSteps = nbSteps
        self._nbSimus = nbSimus
        
        # Regarding the path:
        # The first dimension is the standard path of the asset
        # The second dimension is the antithetic path
        # The third one corresponds to the X_{\cdot} in the paper, i.e. X_{n, \cdot} = \mu_{n-1}(\theta)
        
        self._path = torch.empty(size=(nbSimus, 1, nbSteps), dtype=torch.complex64)
        self._tangentPathS = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathVol = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathRfr = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathSVol = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathVolVol = torch.empty(size=(nbSimus, 1, nbSteps))
        self._gaussians = torch.empty(size=(nbSimus, 1, nbSteps)) # We save the Gaussians for the American option case.
        self._directPath = torch.empty(size=(3, nbSimus, 1, 2))
        
    
    def Simulate(self, startTime, endTime):
        
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
        
        #Reinitializing the paths in case they were already simulated, and initializing the original time step
        self._path = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._path[:, 0, 0] = self._spot # paths initialization to the initial spots
        
        #Initializing the tangent Paths to 1 in the case where we derive wrt the spot.
        self._tangentPathS = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._tangentPathS[:, 0, 0] = 1
        
        #Initializing the tangent Paths to 0 in the case where we derive wrt the vol.
        self._tangentPathVol = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._tangentPathVol[:, 0, 0] = 0
        
        #Initializing the tangent Paths to 0 in the case where we derive wrt the rfr.
        self._tangentPathRfr = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._tangentPathRfr[:, 0, 0] = 0
        
        #Initializing the tangent Paths to 1 in the case where we derive wrt the spot and the vol.
        self._tangentPathSVol = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._tangentPathSVol[:, 0, 0] = 0
        
        #Initializing the tangent Paths to 0 in the case where we derive wrt the vol two times.
        self._tangentPathVolVol = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._tangentPathVolVol[:, 0, 0] = 0
        
        self._gaussians[:, 0, 0] = 0 
        # Generating independent standard gaussians:
        self._gaussians[:, 0, 1:] = torch.tensor(np.random.normal(size = (self._nbSimus, self._nbSteps-1)))
        self._gaussians = torch.tensor(self._gaussians)
        
        for i in range(1, self._nbSteps):
            
            p = self._path[:, 0, i-1].clone()
            tPS = self._tangentPathS[:, 0, i-1].clone()
            tPV = self._tangentPathVol[:, 0, i-1].clone()
            tPR = self._tangentPathRfr[:, 0, i-1].clone()
            tPSV = self._tangentPathSVol[:, 0, i-1].clone()
            tPVV = self._tangentPathVolVol[:, 0, i-1].clone()
            
            self._path[:, 0, i] = p + self._rfr * dt * p + \
                                self._vol * p * torch.sqrt(dt) * self._gaussians[:, 0, i]
            
            self._tangentPathS[:, 0, i] = tPS + self._rfr * dt * tPS + \
                                        tPS * self._vol * torch.sqrt(dt) * self._gaussians[:, 0, i]
            
            self._tangentPathVol[:, 0, i] = tPV + self._rfr * dt * tPV + \
                                            (tPV * self._vol *  torch.sqrt(dt) + p *  torch.sqrt(dt) ) * self._gaussians[:, 0, i]
            
            self._tangentPathRfr[:, 0, i] = tPR + self._rfr * dt * tPR + dt * p + \
                                            tPR * self._vol * torch.sqrt(dt) * self._gaussians[:, 0, i]
        
            self._tangentPathSVol[:, 0, i] = tPSV + self._rfr * tPSV * dt +\
                                            (tPS + self._vol * tPSV) * torch.sqrt(dt) * self._gaussians[:, 0, i]
            
            self._tangentPathVolVol[:, 0, i] = tPVV + self._rfr * tPVV * dt + \
                                            (2 * tPV + self._vol * tPVV)* torch.sqrt(dt) * self._gaussians[:, 0, i]
            


## European Black & Scholes Vibrato

Pour le Vibrato à l'ordre 2, on a les dérivées partielles suivantes:

$\partial_x \mu_{n-1} = Y_{n-1}^x (1 + rh)$, $\partial_x \sigma_{n-1} =  Y_{n-1}^{x} \sigma \sqrt{h}$

$\partial_{\sigma} \mu_{n-1} = Y_{n-1}^{\sigma} (1 + rh)$,   $\partial_{\sigma} \sigma_{n-1} =  Y_{n-1}^{\sigma} \sigma \sqrt{h} + X_{n-1}\sqrt{h}$

$\partial_{xx}\mu_{n-1} = Y_{n-1}^{xx}(1 + rh) = 0$,   $\partial_{xx}\sigma_{n-1} = Y_{n-1}^{xx} \sigma \sqrt{h} = 0$

$\partial_{\sigma \sigma}\mu_{n-1} = Y_{n-1}^{\sigma \sigma}(1 + rh)$,   $\partial_{\sigma \sigma}\sigma_{n-1} = Y_{n-1}^{\sigma \sigma} \sigma \sqrt{h} + 2Y_{n-1}^{\sigma}\sqrt{h}$

$\partial_{\sigma x}\mu_{n-1} = Y_{n-1}^{\sigma x}(1 + rh)$,   $\partial_{\sigma x}\sigma_{n-1} = Y_{n-1}^{\sigma x} \sigma \sqrt{h} + Y_{n-1}^x \sqrt{h}$

On utilise alors une méthode de Monte Carlo en faisant la moyenne empirique des quantités (nous n'avons pas implémenté de méthode antithétique ici):

$\partial_{ij}\mu_{n-1} V_{T}\frac{Z_n}{\sigma \sqrt{h}} + \partial_{i}\mu_{n-1}\partial_{j}\mu_{n-1} V_T \frac{Z_n^2 - 1}{\sigma^2 h } + \partial_{i}\sigma_{n-1}\partial_{j}\sigma_{n-1}V_T\frac{Z_n^4 - 5Z_n^2 + 2}{\sigma^2 h} + \partial_{ij}\sigma_{n-1} V_{T}\frac{Z_n^2 - 1}{\sigma \sqrt{h}} + (\partial_{i}\mu_{n-1}\partial_{j}\sigma_{n-1} + \partial_{j}\mu_{n-1}\partial_{i}\sigma_{n-1})V_T\frac{Z_n^{3} - 3Z_n}{\sigma^2 h}$

avec ici $\sigma = \sigma X_{n-1}$

In [ ]:
class EuropeanVibrato1D:
    
    def __init__(self, payoff, mZ, bs1D, timeStep, isAnti=True):
        """
        parameters:
        - payoff: a payoff object (i.e. just a class with a ComputePayoff method that returns a double).
        - mZ(int): number of Gaussians simulations for the last time step
        - bs1D(BlackScholes1D): needs the paths to be already generated
        - timeStep(int): <= nbSteps de bs1D et est surtout utile dans le cas Américain qui appelle cette classe.
        - isAnti(bool): if True then uses antithetic variate
        """
        self._spot = bs1D._spot
        self._rfr = bs1D._rfr
        self._vol = bs1D._vol
        
        self._nbSteps = bs1D._nbSteps
        self._nbSimus = bs1D._nbSimus
        self._payoff = payoff
        self._payoffs = torch.empty(size=(nbSimus,mZ))
        self._deltaVibrato = torch.empty(size=(nbSimus,mZ))
        self._vegaVibrato = torch.empty(size=(nbSimus,mZ))
        self._rhoVibrato = torch.empty(size=(nbSimus,mZ))
        self._gammaVibrato = torch.empty(size=(nbSimus,mZ))
        self._vegavegaVibrato = torch.empty(size=(nbSimus,mZ))
        self._deltavegaVibrato = torch.empty(size=(nbSimus,mZ))
        
        self._timeStep = timeStep
        
        self._bs1D = bs1D
        self._path = bs1D._path[:, :, :timeStep]
        self._tangentPathS = bs1D._tangentPathS[:, :, :timeStep]
        self._tangentPathVol = bs1D._tangentPathVol[:, :, :timeStep]
        self._tangentPathRfr = bs1D._tangentPathRfr[:, :, :timeStep]
        self._tangentPathSVol = bs1D._tangentPathSVol[:, :, :timeStep]
        self._tangentPathVolVol = bs1D._tangentPathVolVol[:, :, :timeStep]
        self._gaussians = bs1D._gaussians[:, :, :timeStep]
            
        self._mZ = mZ
        
        self._isAnti = isAnti
        
        
    def ComputeVibrato(self, startTime, endTime, strike):
        
        start=time.time()
        
        T = endTime - startTime
        T = torch.tensor(T)
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
        
        strike=torch.tensor(strike)
        
        #Reinitializing the values if we reuse the function for the same vibrato object
        self._payoffs = torch.empty(size=(nbSimus,mZ))
        self._deltaVibrato = torch.empty(size=(nbSimus,mZ))
        self._vegaVibrato = torch.empty(size=(nbSimus,mZ))
        self._rhoVibrato = torch.empty(size=(nbSimus,mZ))
        
        # Generating the M_Z last time steps:
        lastGaussians = np.random.normal(size=(self._nbSimus, self._mZ))
        lastGaussians = torch.tensor(lastGaussians)
        lastSpots = torch.empty(size=(self._nbSimus, self._mZ, 3)) # We have to generate the standard value, the antithetic and the mu part
        payoffs = torch.empty(size=(self._nbSimus, self._mZ))
        payoffsAnti = torch.empty(size=(self._nbSimus, self._mZ))
        payoffsDot = torch.empty(size=(self._nbSimus, self._mZ))
        
        # Computing the first derivative wrt to Theta using Vibrato:
        muDelta = self._tangentPathS[:, 0, -1] * (1 + self._rfr * dt)
        muVega = self._tangentPathVol[:, 0, -1] * (1 + self._rfr * dt)
        muRho = self._tangentPathRfr[:, 0, -1] * (1 + self._rfr * dt) + self._path[:, 0, -1] * dt
        
        sigmaDelta = self._tangentPathS[:, 0, -1] * self._vol * torch.sqrt(dt)
        sigmaVega = self._tangentPathVol[:, 0, -1] * self._vol * torch.sqrt(dt) + self._path[:, 0, -1] * torch.sqrt(dt)
        sigmaRho = self._tangentPathRfr[:, 0, -1] * self._vol * torch.sqrt(dt)
    
        
        for j in range(self._mZ):

            lastSpots[:, j, 0] = self._path[:, 0, -1] * (1 + self._rfr * dt + self._vol * torch.sqrt(dt) * lastGaussians[:, j])
            lastSpots[:, j, 1] = self._path[:, 0, -1] * (1 + self._rfr * dt - self._vol * torch.sqrt(dt) * lastGaussians[:, j])
            lastSpots[:, j, 2] = self._path[:, 0, -1] * (1 + self._rfr * dt)
            
            #payoffs[:, j] = self._payoff.ComputePayoff(lastSpots[:, j, 0])
            #payoffsAnti[:, j] = self._payoff.ComputePayoff(lastSpots[:, j, 1])
            #payoffsDot[:, j] = self._payoff.ComputePayoff(lastSpots[:, j, 2])
            
            payoffs[:, j] = torch.maximum(lastSpots[:, j, 0] - strike, torch.tensor(0))
            payoffsAnti[:, j] = torch.maximum(lastSpots[:, j, 1] - strike, torch.tensor(0))
            payoffsDot[:, j] = torch.maximum(lastSpots[:, j, 2] - strike, torch.tensor(0))
            
            termDelta1 = muDelta * 0.5 * (payoffs[:, j] - payoffsAnti[:, j]) * lastGaussians[:, j]
            termDelta2 = sigmaDelta * 0.5 * (payoffs[:, j] + payoffsAnti[:, j] - 2 * payoffsDot[:, j]) * (torch.square(lastGaussians[:, j]) - 1) 
            
            
            if self._isAnti== False:
                termDelta1 = muDelta * (payoffs[:, j]) * lastGaussians[:, j]
                termDelta2 = sigmaDelta * (payoffs[:, j]) * (torch.square(lastGaussians[:, j]) - 1) 
            
                
            self._deltaVibrato[:, j] = torch.exp(-self._rfr * T) * (termDelta1 + termDelta2)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
            termVega1 = muVega * 0.5 * (payoffs[:, j] - payoffsAnti[:, j]) * lastGaussians[:, j]
            termVega2 = sigmaVega * 0.5 * (payoffs[:, j] + payoffsAnti[:, j] - 2 * payoffsDot[:, j]) * (torch.square(lastGaussians[:, j]) - 1) 
            self._vegaVibrato[:, j] = torch.exp(-self._rfr * T) * (termVega1 + termVega2)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
            termRho1 = muRho * 0.5 * (payoffs[:, j] - payoffsAnti[:, j]) * lastGaussians[:, j]
            termRho2 = sigmaRho * 0.5 * (payoffs[:, j] + payoffsAnti[:, j] - 2 * payoffsDot[:, j]) * (torch.square(lastGaussians[:, j]) - 1) 
            self._rhoVibrato[:, j] = -T * torch.exp(-self._rfr * T) * payoffs[:, j] + \
                                    torch.exp(-self._rfr * T) * (termRho1 + termRho2)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
        
        self._payoffs = 0.5 * torch.exp(-self._rfr * T) * (payoffs + payoffsAnti)
        if self._isAnti==False:
            self._payoffs =torch.exp(-self._rfr * T) * (payoffs)
            
        self._deltaVibrato = torch.mean(self._deltaVibrato, dim=1) # averaging the M_Z part.
        self._vegaVibrato = torch.mean(self._vegaVibrato, dim=1)
        self._rhoVibrato = torch.mean(self._rhoVibrato, dim=1)
        
        end = time.time()
        print(end-start)
        
        
        
    def ComputeSecondOrderVibrato(self, startTime, endTime):
        
        
        start = time.time()
        T = endTime - startTime
        T = torch.tensor(T)
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
        
        #Reinitializing the values if we reuse the function for the same vibrato object
        self._gammaVibrato = torch.empty(size=(nbSimus,mZ))
        self._vegavegaVibrato = torch.empty(size=(nbSimus,mZ))
        self._deltavegaVibrato = torch.empty(size=(nbSimus,mZ))
        
        # Generating the M_Z last time steps:
        lastGaussians = torch.tensor(np.random.normal(size=(self._nbSimus, self._mZ)))
        lastSpots = torch.empty(size=(self._nbSimus, self._mZ)) # We only generate the standard value
        payoffs = torch.empty(size=(self._nbSimus, self._mZ))
        
        # Computing the first derivative wrt to Theta using Vibrato:
        muDelta = self._tangentPathS[:, 0, -1] * (1 + self._rfr * dt)
        muVega = self._tangentPathVol[:, 0, -1] * (1 + self._rfr * dt)
        muVV = self._tangentPathVolVol[:, 0, -1] * (1 + self._rfr * dt)
        muVX = self._tangentPathSVol[:, 0, -1] * (1 + self._rfr * dt) 
        
        sigmaDelta = self._tangentPathS[:, 0, -1] * self._vol * torch.sqrt(dt)
        sigmaVega = self._tangentPathVol[:, 0, -1] * self._vol * torch.sqrt(dt) + self._path[:, 0, -1] * torch.sqrt(dt)
        sigmaVV = self._tangentPathVolVol[:, 0, -1] * self._vol * torch.sqrt(dt) + 2 * self._tangentPathVol[:, 0, -1] * torch.sqrt(dt)
        sigmaVX = self._tangentPathRfr[:, 0, -1] * self._vol * torch.sqrt(dt) +  self._tangentPathS[:, 0, -1] * torch.sqrt(dt)
    
        
        for j in range(self._mZ):

            lastSpots[:, j] = self._path[:, 0, -1] * (1 + self._rfr * dt + self._vol * torch.sqrt(dt) * lastGaussians[:, j])
            payoffs[:, j] = self._payoff.ComputePayoff(lastSpots[:, j])
            
            termSS1 = torch.square(muDelta) * payoffs[:, j] * (torch.square(lastGaussians[:, j])-1)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            termSS2 = torch.square(sigmaDelta) * payoffs[:, j] * (torch.pow(lastGaussians[:, j], 4) - 5*torch.square(lastGaussians[:, j]) + 2)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            termSS3 = 2 * muDelta * sigmaDelta * payoffs[:, j] * (torch.pow(lastGaussians[:, j], 3) - 3 * lastGaussians[:, j])/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
            self._gammaVibrato[:, j] = torch.exp(-self._rfr * T) * (termSS1 + termSS2 + termSS3)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
            termVV1 = muVV * payoffs[:, j] * lastGaussians[:, j]
            termVV2 = torch.square(muVega) * payoffs[:, j] * (torch.square(lastGaussians[:, j])-1)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            termVV3 = torch.square(sigmaVega) * payoffs[:, j] * (torch.pow(lastGaussians[:, j], 4) - 5*torch.square(lastGaussians[:, j]) + 2)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            termVV4 = sigmaVV * payoffs[:, j] * (torch.square(lastGaussians[:, j]) - 1)
            termVV5 = 2 * muVega * sigmaVega * payoffs[:, j] * (torch.pow(lastGaussians[:, j], 3) - 3 * lastGaussians[:, j])/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
            self._vegavegaVibrato[:, j] = torch.exp(-self._rfr * T) * (termVV1 + termVV2 + termVV3 + termVV4 + termVV5)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
            
            termVS1 = muVX * payoffs[:, j] * lastGaussians[:, j]
            termVS2 = muVega * muDelta * payoffs[:, j] * (torch.square(lastGaussians[:, j])-1)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            termVS3 = sigmaVega * sigmaDelta * payoffs[:, j] * (torch.pow(lastGaussians[:, j], 4) - 5*torch.square(lastGaussians[:, j]) + 2)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            termVS4 = sigmaVX * payoffs[:, j] * (torch.square(lastGaussians[:, j]) - 1)
            termVS5 = (muVega * sigmaDelta + muDelta * sigmaVega) * payoffs[:, j] * (torch.pow(lastGaussians[:, j], 3) - 3 * lastGaussians[:, j])/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
            self._deltavegaVibrato[:, j] = torch.exp(-self._rfr * T) * (termVS1 + termVS2 + termVS3 + termVS4 + termVS5)/(self._path[:, 0, -1] * self._vol * torch.sqrt(dt))
            
        self._payoffs = payoffs[:, 0]
            
        self._gammaVibrato = torch.mean(self._gammaVibrato, dim=1) # averaging the M_Z part.
        self._vegavegaVibrato = torch.mean(self._vegavegaVibrato, dim=1)
        self._deltavegaVibrato = torch.mean(self._deltavegaVibrato, dim=1)
  
        end = time.time()
        print(end-start)
  

## Tests pour call européen Black & Scholes

In [ ]:
#paramètres:
strike = 100.
spot = 100.
vol = 0.2
rfr = 0.05
nbSimus = 100000
nbSteps = 20
endTime = 1.
startTime = 0.
T = endTime - startTime
mZ = 1

payoff = CallPayoff(strike)

### Comparing the time complexity and variance between the different methods

first results, spot = 100, spot=90, spot=110

#### Black & Scholes values:

price = 10.4506, Delta = 0.6368, Gamma  = 0.01876

5.091222078817552 0.42983173188954316 0.021819747579660095

17.66295374059044 0.7957541713095866 0.012886510906085861

In [ ]:
bsPrice = CallPrice(spot, rfr, vol, strike, T)
bsDelta = DeltaCall(spot, rfr, vol, strike, T)
bsGamma = GammaCall(spot, rfr, vol, strike, T)
print(bsPrice, bsDelta, bsGamma)

#### With the finite difference method (with antithetic)

for price: {'mean': tensor(10.4358),
 'var': tensor(200.1929),
 'lower': tensor(10.3481),
 'upper': tensor(10.5235)}, time = 0.07895588874816895
 
 for delta: {'mean': tensor(0.6375),
 'var': tensor(0.2946),
 'lower': tensor(0.6342),
 'upper': tensor(0.6409)}, time=0.16090941429138184
 
 for gamma: {'mean': tensor(0.0286),
 'var': tensor(54.0349),
 'lower': tensor(-0.0170),
 'upper': tensor(0.0741)}, 0.22786688804626465
 
 
 SPOT = 90:
 
 {'mean': tensor(4.5192),
 'var': tensor(23.2287),
 'lower': tensor(4.4893),
 'upper': tensor(4.5491)}, 0.005994319915771484
 
 0.013991355895996094
{'mean': tensor(0.4517),
 'var': tensor(0.0666),
 'lower': tensor(0.4501),
 'upper': tensor(0.4533)}
 
 0.015993356704711914
{'mean': tensor(0.0190),
 'var': tensor(18.4971),
 'lower': tensor(-0.0077),
 'upper': tensor(0.0456)}
 
 
 
 SPOT = 110:
 
 {'mean': tensor(17.7173),
 'var': tensor(22.0555),
 'lower': tensor(17.6882),
 'upper': tensor(17.7464)}
 
 {'mean': tensor(0.8178),
 'var': tensor(0.0364),
 'lower': tensor(0.8167),
 'upper': tensor(0.8190)}
 
 {'mean': tensor(-1.2714e-07),
 'var': tensor(8.5076e-13),
 'lower': tensor(-1.3286e-07),
 'upper': tensor(-1.2142e-07)}

In [ ]:
eps = 0.0001
fd1D = FD1D(nbSteps, nbSimus, payoff)

In [ ]:
price = fd1D.MCPrice(startTime, endTime, spot, vol, rfr)
monte_carlo(price)

In [ ]:
delta= fd1D.ComputeDelta(startTime, endTime, spot, vol, rfr, eps)
monte_carlo(delta)

In [ ]:
gamma = fd1D.ComputeGamma(startTime, endTime, spot, vol, rfr, eps)
monte_carlo(gamma)

#### With full second order vibrato

Tout est implémenté avec antithétique et mZ = 1 ici (delta and price same as with VAD)


for the Gamma {'mean': tensor(0.0260, grad_fn=<MeanBackward0>),
 'var': tensor(2.4200, grad_fn=<VarBackward0>),
 'lower': tensor(0.0164, grad_fn=<SubBackward0>),
 'upper': tensor(0.0357, grad_fn=<AddBackward0>)}, time=0.08095479011535645
    
 
    SPOT90:
 
   {'mean': tensor(0.0244, grad_fn=<MeanBackward0>),
 'var': tensor(0.0430, grad_fn=<VarBackward0>),
 'lower': tensor(0.0231, grad_fn=<SubBackward0>),
 'upper': tensor(0.0257, grad_fn=<AddBackward0>)}
    
    
    SPOT110:
    
    {'mean': tensor(0.0112, grad_fn=<MeanBackward0>),
 'var': tensor(0.0610, grad_fn=<VarBackward0>),
 'lower': tensor(0.0096, grad_fn=<SubBackward0>),
 'upper': tensor(0.0127, grad_fn=<AddBackward0>)}

In [ ]:
bs1D = BlackScholes1D(spot, rfr, vol, nbSteps, nbSimus)

In [ ]:
bs1D.Simulate(startTime, endTime)

In [ ]:
vib = EuropeanVibrato1D(payoff, mZ, bs1D, timeStep=nbSteps)

In [ ]:
vib.ComputeSecondOrderVibrato(startTime, endTime)

In [ ]:
monte_carlo(vib._gammaVibrato)

In [ ]:
vib.ComputeVibrato(startTime, endTime, strike)
monte_carlo(vib._deltaVibrato)

#### With VAD

AVEC ANTITHETIQUE ET MZ = 2:

{'mean': tensor(10.4372, grad_fn=<MeanBackward0>),
 'var': tensor(198.7163, grad_fn=<VarBackward0>),
 'lower': tensor(10.3498, grad_fn=<SubBackward0>),
 'upper': tensor(10.5246, grad_fn=<AddBackward0>)}
    
{'mean': tensor(0.6377, grad_fn=<MeanBackward0>),
 'var': tensor(0.9574, grad_fn=<VarBackward0>),
 'lower': tensor(0.6316, grad_fn=<SubBackward0>),
 'upper': tensor(0.6437, grad_fn=<AddBackward0>)}, temps= 0.07895278930664062

 gamma mean = 0.0187, temps = 1.925095558166504

    
AVEC VARIABLE ANTITHETIQUE:

For the price: {'mean': tensor(10.4778, grad_fn=<MeanBackward0>),
 'var': tensor(200.0261, grad_fn=<VarBackward0>),
 'lower': tensor(10.3901, grad_fn=<SubBackward0>),
 'upper': tensor(10.5654, grad_fn=<AddBackward0>)}
    
{'mean': tensor(0.6428, grad_fn=<MeanBackward0>),
 'var': tensor(1.6715, grad_fn=<VarBackward0>),
 'lower': tensor(0.6348, grad_fn=<SubBackward0>),
 'upper': tensor(0.6508, grad_fn=<AddBackward0>)}temps=0.038585662841796875
    
For the Gamma: {mean = 0.0186}, temps=1.764327049255371.
    
Sadly, we were not able to get a confidence interval for the Gamma due to the nature of pytorch autograd only allowing for a scalar output.


SANS ANTITHETIQUE:

{'mean': tensor(10.4320, grad_fn=<MeanBackward0>),
 'var': tensor(212.4292, grad_fn=<VarBackward0>),
 'lower': tensor(10.3417, grad_fn=<SubBackward0>),
 'upper': tensor(10.5224, grad_fn=<AddBackward0>)} 0.036980628967285156
    
For the Delta: {'mean': tensor(.6795, grad_fn=<MeanBackward0>),
 'var': tensor(2.8479, grad_fn=<VarBackward0>),
 'lower': tensor(0.6690, grad_fn=<SubBackward0>),
 'upper': tensor(0.6900, grad_fn=<AddBackward0>)} 

  
    gamma = 0.0180, temps=1.634746789932251
    
 POUR LES AUTRES VALEURS DE SPOT (avec antithétique et mZ=1)
    
    
    SPOT90:
    
    {'mean': tensor(5.0770, grad_fn=<MeanBackward0>),
 'var': tensor(93.2431, grad_fn=<VarBackward0>),
 'lower': tensor(5.0171, grad_fn=<SubBackward0>),
 'upper': tensor(5.1368, grad_fn=<AddBackward0>)}
    
    {'mean': tensor(0.4355, grad_fn=<MeanBackward0>),
 'var': tensor(1.2313, grad_fn=<VarBackward0>),
 'lower': tensor(0.4286, grad_fn=<SubBackward0>),
 'upper': tensor(0.4424, grad_fn=<AddBackward0>)}
    
    gamma = 0.0220
    
    
    SPOT110:
    
    {'mean': tensor(17.7075, grad_fn=<MeanBackward0>),
 'var': tensor(333.6004, grad_fn=<VarBackward0>),
 'lower': tensor(17.5943, grad_fn=<SubBackward0>),
 'upper': tensor(17.8207, grad_fn=<AddBackward0>)}
    
    
    {'mean': tensor(0.7933, grad_fn=<MeanBackward0>),
 'var': tensor(1.8671, grad_fn=<VarBackward0>),
 'lower': tensor(0.7849, grad_fn=<SubBackward0>),
 'upper': tensor(0.8018, grad_fn=<AddBackward0>)}
    
    gamma = tensor(0.0127)

In [ ]:
bs1D = BlackScholes1D(spot, rfr, vol, nbSteps, nbSimus)

In [ ]:
bs1D.Simulate(startTime, endTime)

In [ ]:
vib = EuropeanVibrato1D(payoff, mZ, bs1D, timeStep=nbSteps, isAnti=True)
vib.ComputeVibrato(startTime, endTime, strike)

In [ ]:
start = time.time()
out = vib._deltaVibrato.mean()
out.backward()
gamma = bs1D._spot.grad
end = time.time()
print(end-start)

In [ ]:
gamma

In [ ]:
monte_carlo(vib._payoffs)

In [ ]:
monte_carlo(vib._deltaVibrato)

# III- Case of a European Call in the Heston Model

## Heston path generation

In [ ]:
class Heston1D:
    
    def __init__(self, spot, rfr, vol, correl, kappa, eta, xi, nbSteps, nbSimus):
        """
        parameters:
        - spot(array): initial value of the underlying
        - rfr(double): risk free rate
        - vol(array): Black & Scholes volatility of the underlying
        - correl(double): correlation between the two brownian motions.
        - nbSteps(int): number of discretization steps
        - nbSimus(int): nb of Monte Carlo simulations
        """
        self._spot = torch.tensor(spot, requires_grad=True)
        self._rfr = torch.tensor(rfr, requires_grad=True)
        self._vol = torch.tensor(vol, requires_grad=True)
        self._correl = correl
        self._kappa = kappa
        self._eta = eta
        self._xi = xi
        self._nbSteps = nbSteps
        self._nbSimus = nbSimus
        
        # Regarding the path:
        # The first dimension is the standard path of the asset
        # The second dimension is the antithetic path
        # The third one corresponds to the X_{\cdot} in the paper, i.e. X_{n, \cdot} = \mu_{n-1}(\theta)
        
        self._pathAsset = torch.empty(size=(nbSimus, 1, nbSteps))
        self._pathVariance = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathS = torch.empty(size=(nbSimus, 1, nbSteps))
        self._gaussians = torch.empty(size=(nbSimus, 2, nbSteps)) 
        
    
    def Simulate(self, startTime, endTime):
        
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
       # W = Brownian1D(self._nbSteps, self._nbSimus)
       # W.Simulate(startTime, endTime)
       # self._gaussians = W._gaussians
        
        #Reinitializing the paths in case they were already simulated, and initializing the original time step
        self._path = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._path[:, 0, 0] = self._spot # paths initialization to the initial spots
        
        #Reinitializing the paths in case they were already simulated, and initializing the original time step
        self._pathVariance = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._pathVariance[:, 0, 0] = self._vol # paths initialization to the initial variance
        
        #Initializing the tangent Paths to 1 in the case where we derive wrt the spot.
        self._tangentPathS = torch.empty(size=(self._nbSimus, 1, self._nbSteps))
        self._tangentPathS[:, 0, 0] = 1
        
        self._gaussians[:, :, 0] = 0 
        # Generating independent standard gaussians:
        gaussTemp = np.random.normal(size = (self._nbSimus, 2, self._nbSteps-1))
        self._gaussians[:, :, 1:] = torch.tensor(gaussTemp.astype(np.float32))
        
        matGauss = np.array([[1, 0], [self._correl, torch.sqrt(1 - torch.square(torch.tensor(self._correl)))]])
        matGauss = torch.tensor(matGauss, dtype=torch.float64)
        
        
        for i in range(1, self._nbSteps):
            
            p = self._path[:, 0, i-1].clone()
            pV = self._pathVariance[:, 0, i-1].clone()
            tPS = self._tangentPathS[:, 0, i-1].clone()
            
            gaussTemp2 = torch.reshape(self._gaussians[:, :, i], shape=(self._nbSimus, 2, 1))
            gaussTemp2 = torch.tensor(gaussTemp2, dtype=torch.float64)
            
            gaussianCorr = matGauss @ gaussTemp2
            
            self._path[:, 0, i] = p + self._rfr * dt * p + torch.sqrt(pV) * p * torch.sqrt(dt) * gaussianCorr[:, 0, 0]
            
            self._pathVariance[:, 0, i] = pV +self._kappa * (self._eta - pV) * dt + \
                                            self._xi * torch.sqrt(pV * dt) *  gaussianCorr[:, 1, 0]
            
            self._tangentPathS[:, 0, i] = tPS + self._rfr * dt * tPS + tPS * torch.sqrt(pV * dt) * gaussianCorr[:, 0, 0]
            
            

## Vibrato for Heston

In [ ]:
class EuropeanHestonVibrato1D:
    
    def __init__(self, spot, rfr, vol, correl, kappa, eta, xi, nbSteps, nbSimus, payoff, mZ, heston1D):
        """
        parameters:
        - spots(array): initial values of the underlying
        - rfr(double): risk free rate
        - vol(array): Black & Scholes volatility of the underlying
        - nbSteps(int): number of discretization steps
        - nbSimus(int): nb of Monte Carlo simulations
        - payoff: a payoff object (i.e. just a class with a ComputePayoff method that returns a double).
        - mZ(int): number of Gaussians simulations for the last time step
        """
        self._spot = torch.tensor(spot, requires_grad=True)
        self._rfr = torch.tensor(rfr, requires_grad=True)
        self._vol = torch.tensor(vol, requires_grad=True)
        self._correl = correl
        self._kappa = kappa
        self._eta = eta
        self._xi = xi
        self._nbSteps = nbSteps
        self._nbSimus = nbSimus
        self._payoff = payoff
        self._deltaVibrato = torch.empty(size=(nbSimus,mZ))
        self._vegaVibrato = torch.empty(size=(nbSimus,mZ))
        self._rhoVibrato = torch.empty(size=(nbSimus,mZ))
        self._mZ = mZ
        self._heston1D = heston1D
        self._payoffs = torch.empty(size=(self._nbSimus, self._mZ))
        self._payoffsAnti = torch.empty(size=(self._nbSimus, self._mZ))
        
        
        
    def ComputeVibrato(self, startTime, endTime, strike):
        
        start = time.time()
        
        T = endTime - startTime
        T = torch.tensor(T)
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(T)
        
        strike = torch.tensor(strike)
        
        # Generating the M_Z last time steps for the underlying
        lastGaussians = np.random.normal(size=(self._nbSimus, self._mZ))
        lastGaussians = torch.tensor(lastGaussians)
        lastSpots = torch.empty(size=(self._nbSimus, self._mZ, 3)) # We have to generate the standard value, the antithetic and the mu part
        payoffs = torch.empty(size=(self._nbSimus, self._mZ))
        payoffsAnti = torch.empty(size=(self._nbSimus, self._mZ))
        payoffsDot = torch.empty(size=(self._nbSimus, self._mZ))
        
        # Computing the first derivative wrt to Theta using Vibrato:
        muDelta = self._heston1D._tangentPathS[:, 0, -1] * (1 + self._rfr * dt)
        
        sigmaDelta = self._heston1D._tangentPathS[:, 0, -1] * torch.sqrt(self._heston1D._pathVariance[:, 0, -1] * dt)
    
        
        for j in range(self._mZ):

            lastSpots[:, j, 0] = self._heston1D._path[:, 0, -1] * (1 + self._rfr * dt + torch.sqrt(heston1D._pathVariance[:, 0, -1] * dt) * lastGaussians[:, j])
            lastSpots[:, j, 1] = self._heston1D._path[:, 0, -1] * (1 + self._rfr * dt - torch.sqrt(heston1D._pathVariance[:, 0, -1] * dt) * lastGaussians[:, j])
            lastSpots[:, j, 2] = self._heston1D._path[:, 0, -1] * (1 + self._rfr * dt)
            #payoffs[:, j] = self._payoff.ComputePayoff(lastSpots[:, j, 0])
            #payoffsAnti[:, j] = self._payoff.ComputePayoff(lastSpots[:, j, 1])
            #payoffsDot[:, j] = self._payoff.ComputePayoff(lastSpots[:, j, 2])
            
            payoffs[:, j] = torch.maximum(lastSpots[:, j, 0] - strike, torch.tensor(0))
            payoffsAnti[:, j] = torch.maximum(lastSpots[:, j, 1] - strike, torch.tensor(0))
            payoffsDot[:, j] = torch.maximum(lastSpots[:, j, 2] - strike, torch.tensor(0))
            
            termDelta1 = muDelta * 0.5 * (payoffs[:, j] - payoffsAnti[:, j]) * lastGaussians[:, j]
            termDelta2 = sigmaDelta * 0.5 * (payoffs[:, j] + payoffsAnti[:, j] - 2 * payoffsDot[:, j]) * (np.square(lastGaussians[:, j]) - 1) 
            self._deltaVibrato[:, j] = torch.exp(-self._rfr * T) * (termDelta1 + termDelta2)/(self._heston1D._path[:, 0, -1] * torch.sqrt(self._heston1D._pathVariance[:, 0, -1] * dt))
            
            
        self._deltaVibrato = torch.mean(self._deltaVibrato, dim=1) #averaging the M_Z part.
        self._payoffs = torch.exp(-self._rfr * T) * 0.5 * (payoffs + payoffsAnti)
    
        end = time.time()
        print(end-start)

## Tests results

#### Heston parameters:

In [ ]:
hestonVol = 0.028
correl = 0.5
kappa = 2.93
eta = 0.101
xi = 0.01

#paramètres:
strike = 100.
spot = 100.
vol = 0.2
rfr = 0.05
nbSimus = 100000
nbSteps = 10
endTime = 1.
startTime = 0.
T = endTime - startTime
mZ = 1

payoff = CallPayoff(strike)

#### With Finite Differences

For the price: {'mean': tensor(12.7195),
 'var': tensor(296.3712),
 'lower': tensor(12.6128),
 'upper': tensor(12.8262)}, temps = 0.0819542407989502
 
 For the Delta: {'mean': tensor(0.6300),
 'var': tensor(0.2989),
 'lower': tensor(0.6266),
 'upper': tensor(0.6334)}, temps=0.16590452194213867
 
 For the Gamma: {'mean': tensor(1.0486e-05),
 'var': tensor(5.3198e-09),
 'lower': tensor(1.0034e-05),
 'upper': tensor(1.0938e-05)}, temps= 0.23269295692443848

In [ ]:
eps = 0.00001

In [ ]:
fd1DH = FD1DHeston(nbSteps, nbSimus, payoff)

In [ ]:
price = fd1DH.MCPrice(startTime, endTime, spot, hestonVol, rfr, correl, kappa, eta, xi)
monte_carlo(price)

In [ ]:
delta = fd1DH.ComputeDelta(startTime, endTime, spot, hestonVol, rfr, correl, kappa, eta, xi, eps)
monte_carlo(delta)

In [ ]:
gamma = fd1DH.ComputeGamma(startTime, endTime, spot, hestonVol, rfr, correl, kappa, eta, xi, eps)
monte_carlo(gamma)

#### With VAD

For the price: {'mean': tensor(21.3850, grad_fn=<MeanBackward0>),
 'var': tensor(481.0815, grad_fn=<VarBackward0>),
 'lower': tensor(21.2491, grad_fn=<SubBackward0>),
 'upper': tensor(21.5210, grad_fn=<AddBackward0>)}

for the delta: {'mean': tensor(0.7223, grad_fn=<MeanBackward0>),
 'var': tensor(1.7253, grad_fn=<VarBackward0>),
 'lower': tensor(0.7142, grad_fn=<SubBackward0>),
 'upper': tensor(0.7304, grad_fn=<AddBackward0>)}
0.028983116149902344
    
for the gamma: mean=0.092, temps= 0.7337541580200195

In [ ]:
heston1D = Heston1D(spot, rfr, hestonVol, correl, kappa, eta, xi, nbSteps, nbSimus)
heston1D.Simulate(startTime, endTime)

In [ ]:
vib = EuropeanHestonVibrato1D(spot, rfr, hestonVol, correl, kappa, eta, xi, nbSteps, nbSimus, payoff, mZ, heston1D)

In [ ]:
vib.ComputeVibrato(startTime, endTime, strike)

In [ ]:
monte_carlo(vib._deltaVibrato)

In [ ]:
monte_carlo(vib._payoffs)

In [ ]:
del out

In [ ]:
start = time.time()
out = vib._deltaVibrato.mean()
out.backward()
gamma = heston1D._spot.grad
end = time.time()
print(end-start)

In [ ]:
gamma

# IV- American Vibrato in Black & Scholes model

### ATTENTION: La multiplication de matrices dans Longstaff-Schwartz peut faire crasher le notebook si le nombre de simulation est trop grand

In [ ]:
class AmericanVibrato1D:
    
    def __init__(self, spot, rfr, vol, nbSteps, nbSimus, payoff, m, bs1D, isAnti=True):
        """
        parameters:
        - spots(array): initial values of the underlying
        - rfr(double): risk free rate
        - vol(array): Black & Scholes volatility of the underlying
        - nbSteps(int): number of discretization steps
        - nbSimus(int): nb of Monte Carlo simulations
        - payoff: a payoff object (i.e. just a class with a ComputePayoff method that returns a double
        - m(int): number of functions used to approximate the conditional expectation
        """
        
        self._spot = torch.tensor(spot, requires_grad=True)
        self._rfr = torch.tensor(rfr, requires_grad=True)
        self._vol = torch.tensor(vol, requires_grad=True)
        self._nbSteps = nbSteps
        self._nbSimus = nbSimus
        self._payoff = payoff
        
        self._m = m
        
        
        self._bs1D = bs1D
    
        
        self._path = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathS = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathVol = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathRfr = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathSVol = torch.empty(size=(nbSimus, 1, nbSteps))
        self._tangentPathVolVol = torch.empty(size=(nbSimus, 1, nbSteps))
        self._gaussians = torch.empty(size=(nbSimus, 1, nbSteps))
        
        
        self._optimalPayoffs = torch.empty(size=(nbSimus, 1))
        self._optimalGammas = torch.empty(size=(nbSimus,1))
        
        self._isAnti=isAnti
        
        
    def GetRegressorMatrix(self, spots):
        """
        Function to get the regressor matrix using a polynomial basis.
        """
        regressorMat = torch.empty(size=(self._nbSimus, self._m+1))
        
        for i in range(self._m+1):
            regressorMat[:, i] = torch.pow(spots, i)
                
        return regressorMat
        
    
    def ComputeVibrato(self, startTime, endTime):
        """
        Computation of the Gamma of an American option using Vibrato for first and second order derivatives
        """
        
        print('Launching American Vibrato')
        T = endTime - startTime
        T = torch.tensor(T)
        dt = T/self._nbSteps
        dt = torch.tensor(dt)
        vibrato1D = EuropeanVibrato1D(self._payoff, 1,
                                      bs1D, timeStep = self._nbSteps, isAnti=self._isAnti)
        vibrato1D.ComputeSecondOrderVibrato(startTime, endTime) # Computation of the terminal Gammas for each simulation.
        
        self._path = vibrato1D._path
        self._tangentPathS = vibrato1D._tangentPathS
        self._tangentPathVol = vibrato1D._tangentPathVol
        self._tangentPathRfr = vibrato1D._tangentPathRfr
        self._tangentPathSVol = vibrato1D._tangentPathSVol
        self._tangentPathVolVol = vibrato1D._tangentPathVolVol
        self._gaussians = vibrato1D._gaussians
        
        
        self._optimalPayoffs = torch.exp(-self._rfr * T) * vibrato1D._payoffs
        
        optGamma = torch.exp(-self._rfr * T) * vibrato1D._gammaVibrato.mean()
        self._optimalGammas = torch.full(size=(self._nbSimus, 1), fill_value=optGamma.item())
        
        
        
        for i in range(1, self._nbSteps):
            
            
            #Récupération des spots pour cette time step
            spots = bs1D._path[:, 0, -(i)]
            #Calcul des fonctions de "continuation"
            matX = self.GetRegressorMatrix(spots)
            theta = torch.linalg.inv(matX.T @ matX) @ matX.T @ self._optimalPayoffs
            theta = torch.tensor(theta)
            valueApprox = matX @ theta
            print(valueApprox.shape)
            
            
            #Calcul des Gammas pour cette time step par full vibrato
            vibrato1D = EuropeanVibrato1D(self._payoff,1,bs1D, self._nbSteps - i, isAnti=self._isAnti)
            vibrato1D.ComputeSecondOrderVibrato(startTime, T - i * T/self._nbSteps)
            
            gamma = torch.exp(-self._rfr * T) * vibrato1D._gammaVibrato.mean()
            gammas = torch.full(size=(self._nbSimus, 1), fill_value=gamma.item())
            
            payoffs = torch.exp(-self._rfr * T * (self._nbSteps - i)/self._nbSteps) * self._payoff.ComputePayoff(spots)
            
            for j in range(self._nbSimus):
                if payoffs[j] >= valueApprox[j]:
                    self._optimalPayoffs[j] = payoffs[j]
                    self._optimalGammas[j] = gammas[j]
        
        
        
    def ComputeVAD(self, startTime, endTime, strike):
        """
        Computation of the Gamma of an American option using VAD.
        """
        start = time.time()
        sumGammas = 0
        print('Launching American VAD')
        T = endTime - startTime
        T = torch.tensor(T)
        dt = T/self._nbSteps
        dt = torch.tensor(dt)
        vibrato1D = EuropeanVibrato1D(self._payoff, 1,
                                     bs1D, timeStep=self._nbSteps, isAnti=self._isAnti)
        vibrato1D.ComputeVibrato(startTime, endTime, strike) # Computation of the terminal Deltas
        
        #Computation of the Gamma term for each path by Automatic Differentiation
        out = vibrato1D._deltaVibrato.mean()
        out.backward(retain_graph=True)
        optGamma = torch.exp(-self._rfr * T) * bs1D._spot.grad
        sumGammas+=optGamma
        
        self._optimalGammas = torch.full(size=(self._nbSimus, 1), fill_value=optGamma.item())
        
        
        self._path = vibrato1D._path
        self._tangentPathS = vibrato1D._tangentPathS
        self._tangentPathVol = vibrato1D._tangentPathVol
        self._tangentPathRfr = vibrato1D._tangentPathRfr
        self._tangentPathSVol = vibrato1D._tangentPathSVol
        self._tangentPathVolVol = vibrato1D._tangentPathVolVol
        self._gaussians = vibrato1D._gaussians
        
        
        self._optimalPayoffs = torch.exp(-self._rfr * T) * vibrato1D._payoffs
        
        del out, vibrato1D
        #print(out)
        
        for i in range(1, self._nbSteps):
            
            
            print(f'at step {i}')
            
            #Récupération des spots pour cette time step
            spots = self._path[:, 0, -i]
            #Calcul des fonctions de "continuation"
            matX = self.GetRegressorMatrix(spots)
            theta = torch.linalg.inv(matX.T @ matX) @ matX.T @ self._optimalPayoffs
            theta = torch.tensor(theta)
            valueApprox = matX @ theta
            print(valueApprox.shape)
            print(self._spot)
            
            #Calcul des Gammas pour cette time step par VAD
            vibrato1D = EuropeanVibrato1D(self._payoff,
                                          1, bs1D, timeStep = self._nbSteps-i, isAnti=self._isAnti)
            vibrato1D.ComputeVibrato(startTime, endTime - i * endTime/self._nbSteps, strike)
            out = vibrato1D._deltaVibrato.mean()
            out.backward(retain_graph=True)
            gamma=torch.exp(-self._rfr * T) * bs1D._spot.grad - sumGammas
            sumGammas+=gamma
            gammas = torch.full(size=(self._nbSimus, 1), fill_value=gamma.item())
            
            payoffs = torch.exp(-self._rfr * T * (self._nbSteps - i)/self._nbSteps) * \
                   torch.maximum(spots - torch.tensor(strike), torch.tensor(0))#self._payoff.ComputePayoff(spots)
            
            for j in range(self._nbSimus):
                if payoffs[j] >= valueApprox[j]:
                    self._optimalPayoffs[j] = payoffs[j]
                    self._optimalGammas[j] = gammas[j]
        
        end = time.time()
        print(end-start)

## Tests for an American Option

Here we choose an Americal Call. The reason for this choice is that the quantities computed are supposed to be the same as for a European call, thus allowing for comparison of the results.

We reset the number of simulations to 1000 here as for $M=100 000$ the matrix inversion is too costly on our computer and makes the notebook crash

In [ ]:
spot=100.
m=4
nbSimus=100000
eps=0.0001

#### With finite differences

For the price: {'mean': tensor(10.2130),
 'var': tensor(189.7387),
 'lower': tensor(10.1276),
 'upper': tensor(10.2984)}, temps=22.32023000717163
 
 for the delta: {{'mean': tensor(0.8433),
 'var': tensor(0.1193),
 'lower': tensor(0.8412),
 'upper': tensor(0.8455)}, temps = 55.7230676454533
 
 for the gamma: we find extremely high values (on the order of 2000), we suppose that it is mainly due to the sensitivity to the value of the shock used


S90:

{'mean': tensor(4.5269),
 'var': tensor(67.4162),
 'lower': tensor(4.4761),
 'upper': tensor(4.5778)}
 
 {'mean': tensor(0.4498),
 'var': tensor(0.3371),
 'lower': tensor(0.4462),
 'upper': tensor(0.4534)}
 
 {'mean': tensor(-2.9903e-09),
 'var': tensor(4.5507e-13),
 'lower': tensor(-7.1713e-09),
 'upper': tensor(1.1908e-09)}
 
 S110:
 
 {'mean': tensor(20.7181),
 'var': tensor(195.3728),
 'lower': tensor(20.6315),
 'upper': tensor(20.8047)}
 
 {'mean': tensor(0.4498),
 'var': tensor(0.3371),
 'lower': tensor(0.4462),
 'upper': tensor(0.4534)}
 
 {'mean': tensor(-2.9903e-09),
 'var': tensor(4.5507e-13),
 'lower': tensor(-7.1713e-09),
 'upper': tensor(1.1908e-09)}

In [ ]:
priceLS = fd1D.MCPriceLS(startTime, endTime, spot, vol, rfr)
monte_carlo(priceLS)

In [ ]:
deltaLS = fd1D.ComputeDeltaAmerican(startTime, endTime, spot, vol, rfr, eps)
monte_carlo(deltaLS)

In [ ]:
gammaLS = fd1D.ComputeGammaAmerican(startTime, endTime, spot, vol, rfr, eps)
monte_carlo(gammaLS)

#### With VAD

We don't care about the price here as it is computed by a simple Monte Carlo so the result is equivalent to the one obtained with the finite difference class.


{'mean': tensor(10.0402, grad_fn=<MeanBackward0>),
 'var': tensor(166.9155, grad_fn=<VarBackward0>),
 'lower': tensor(9.9602, grad_fn=<SubBackward0>),
 'upper': tensor(10.1203, grad_fn=<AddBackward0>)}

for the gamma: mean = 'mean': tensor(0.0197), temps=0.2771921157836914
    
S90
    
    {'mean': tensor(4.8131, grad_fn=<MeanBackward0>),
 'var': tensor(73.4427, grad_fn=<VarBackward0>),
 'lower': tensor(4.7600, grad_fn=<SubBackward0>),
 'upper': tensor(4.8662, grad_fn=<AddBackward0>)}
    
    
  gamma: {'mean': tensor(0.0229),
 'var': tensor(4.4066e-06),
 'lower': tensor(0.0228),
 'upper': tensor(0.0229)}
    
S110: 
    
    {'mean': tensor(16.8311, grad_fn=<MeanBackward0>),
 'var': tensor(19.3982, grad_fn=<VarBackward0>),
 'lower': tensor(16.5581, grad_fn=<SubBackward0>),
 'upper': tensor(17.1041, grad_fn=<AddBackward0>)}
    
    gamma=tensor(0.0106)

In [ ]:
bs1D = BlackScholes1D(spot, rfr, vol, nbSteps, nbSimus)
bs1D.Simulate(startTime, endTime)

In [ ]:
amVib1D = AmericanVibrato1D(spot, rfr, vol, nbSteps, nbSimus, payoff, m, bs1D, isAnti=True)

In [ ]:
amVib1D.ComputeVAD(startTime, endTime, strike)

In [ ]:
monte_carlo(amVib1D._optimalGammas)


In [ ]:
monte_carlo(amVib1D._optimalPayoffs)

# V - Black & Scholes multi-dimensional

In [ ]:
class BlackScholesND:
    
    def __init__(self, dim, spots, rfr, corrMat, vols, nbSimus):
        """
        parameters:
        - dim(int): dimension of the underlying
        - spots(array): initial values of the underlying
        - rfr(double): risk free rate
        - corrMat(array): correlation matrix of the underlyings
        - vols(array): array of the Black & Scholes volatilities of the underlyings
        - nbSteps(int): number of discretization steps
        - nbSimus(int): nb of Monte Carlo simulations
        """
        self._dim = torch.tensor(dim)
        self._spots = torch.tensor(spots.astype(np.float32), requires_grad=True)
        self._rfr = torch.tensor(rfr, requires_grad=True)
        self._corrMat = torch.tensor(corrMat)
        self._vols =  torch.tensor(vols, requires_grad=True)
        self._covMat = torch.diagflat(vols) @ corrMat @ torch.diagflat(vols)
        self._nbSimus = nbSimus
        self._paths = torch.empty(size=(nbSimus, dim, 2, 3))
        self._gaussians = torch.empty(size=(nbSimus, dim, 1))
        self._chol = np.empty(shape=(dim, dim))
      
        
    
    def Simulate(self, startTime, endTime):
        
        T = endTime- startTime
        T = torch.tensor(T)
        self._paths = torch.empty(size=(self._nbSimus, self._dim, 2, 3))
        self._paths[:, :, 0, 0] = self._spots # paths initialization to the initial spots
        self._gaussians = np.random.normal(size = (self._nbSimus, self._dim, 1))
        self._gaussians = torch.tensor(self._gaussians.astype(np.float32))
        
        if np.linalg.det(self._covMat)!=0:
            self._chol = np.linalg.cholesky(self._covMat)
            
        else:
            D, P = np.linalg.eig(self._covMat)
            self._chol = P @ np.sqrt(np.diag(D))
            
        self._chol = torch.tensor(self._chol.astype(np.float32))
        
        
        gaussianTerm = torch.sqrt(T) * torch.diagflat(self._spots) @ self._chol @ self._gaussians
        gaussianTerm = torch.reshape(gaussianTerm, shape=(self._nbSimus, self._dim))
        
        p00 =  self._paths[:, :, 0, 0].clone()
        self._paths[:, :, 1, 2] = p00* (1 + self._rfr * T)
        
        p12 = self._paths[:, :, 1, 2].clone()
        
        self._paths[:, :, 1, 0] = p12 + gaussianTerm
        self._paths[:, :, 1, 1] = p12 - gaussianTerm

In [ ]:
class EuropeanVibratoND:
    
    def __init__(self, payoff, bsND):
        """
        parameters:
        - dim(int): dimension of the underlying
        - spots(array): initial values of the underlying
        - rfr(double): risk free rate
        - corrMat(array): correlation matrix of the underlyings
        - vols(array): array of the Black & Scholes volatilities of the underlyings
        - nbSteps(int): number of discretization steps
        - nbSimus(int): nb of Monte Carlo simulations
        """
        self._bsND = bsND
        self._dim = bsND._dim
        self._spots = bsND._spots
        self._rfr = bsND._rfr
        self._corrMat = bsND._corrMat
        self._vols = bsND._vols
        self._nbSimus = bsND._nbSimus
        self._payoff = payoff
        self._deltaVibratoIndiv = torch.empty(size=(nbSimus,dim))
        self._deltaVibrato = torch.empty(size=(nbSimus,))
        self._payoffs = torch.empty(size=(nbSimus,))
        
    
    def ComputeVibrato(self, startTime, endTime, strike, weights):
        
        start=time.time()
        T = endTime - startTime
        T = torch.tensor(T)
        strike = torch.tensor(strike)
        weights = torch.tensor(weights)
        weights = torch.reshape(weights, shape=(self._dim, 1))
        
        lastSpots = self._bsND._paths[:, :, -1, :]
        
        #payoffs = torch.empty(size=(self._nbSimus,))
        #payoffsAnti = torch.empty(size=(self._nbSimus,))
        #payoffsDot = torch.empty(size=(self._nbSimus,))
        payoffs = torch.maximum(lastSpots[:, :, 0] @ weights- strike, torch.tensor(0))
        payoffsAnti = torch.maximum(lastSpots[:, :, 1]@ weights - strike, torch.tensor(0))
        payoffsDot = torch.maximum(lastSpots[:, :, 2]@ weights - strike, torch.tensor(0))
        #payoffs = self._payoff.ComputePayoff(lastSpots[:, :, 0])
        #payoffsAnti = self._payoff.ComputePayoff(lastSpots[:, :, 1])
        #payoffsDot = self._payoff.ComputePayoff(lastSpots[:, :, 2])
        self._payoffs= torch.exp(-self._rfr * T) * 0.5 * (payoffs + payoffsAnti)
        
        
        Z = self._bsND._gaussians
        I = np.identity(self._dim)
        I = torch.tensor(I.astype(np.float32))
        
        zMat = torch.empty(size=(self._nbSimus,self._dim,self._dim))
        sigma = torch.diagflat(self._spots) @ bsND._chol #bsND._covMat #bsND._chol
        invSigma = torch.linalg.inv(sigma)
        
        for i in range(self._nbSimus):
            zTemp = Z[i, :, :]
            zTemp = torch.reshape(zTemp, shape=(self._dim, 1))
            zMat[i, :, :] = zTemp @ zTemp.T
        
        #partialSigma = np.zeros(shape=(self._dim, self._dim, self._dim))
    
        
        for i in range(self._dim):
            partialSigma = np.zeros(shape=(self._dim, self._dim))
            partialSigma = torch.tensor(partialSigma.astype(np.float32))
            partialSigma[i, :] = bsND._chol[i, :]
            prodSigmaZ = invSigma @ Z 
            term1 = (payoffs - payoffsAnti) * (1 + self._rfr * T) * prodSigmaZ[:, i] /(2 * torch.sqrt(T))
            
            tempVar1 = 2 * T * sigma @ partialSigma
            zMat = torch.tensor(zMat, dtype=torch.float32)
            tempVar2 = invSigma @ (zMat - I) @ invSigma
            tempVar3 = tempVar1 @ tempVar2
            tempVar4 = tempVar3.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1)
            #tempVar4 = torch.vmap(torch.trace)(tempVar3)#torch.trace(tempVar3)#, axis1=1, axis2=2)
            tempVar4 = torch.tensor(tempVar4)
            tempVar5 = (payoffs - 2 * payoffsDot + payoffsAnti)
            tempVar5 = torch.reshape(tempVar5, shape=(self._nbSimus,))
            term2 = tempVar5 * tempVar4/(4 * T)
            
            self._deltaVibratoIndiv[:, i] = torch.exp(-self._rfr * T) * (torch.reshape(term1, shape=(self._nbSimus, )) + term2)
            
            del partialSigma, tempVar1, tempVar2, tempVar3, term1, term2
            
        self._deltaVibrato = self._deltaVibratoIndiv @ weights
       # self._deltaVibrato =torch.tensor(np.average(self._deltaVibratoIndiv.numpy(), axis=1, weights=np.reshape(self._payoff._weights.numpy(), newshape=(self._dim,))))
        
        end=time.time()
        print(end-start)
        
     

### Finite difference for Black Scholes ND

In [ ]:
class FDND:
    
    def __init__(self, dim, nbSimus, payoff, weights, strike):
        
        self._dim = dim
        self._payoff = payoff
        self._nbSteps=1
        self._nbSimus = nbSimus
        self._payoffs = torch.empty(size=(nbSimus,1))
        self._gaussians = np.random.normal(size = (self._nbSimus, self._dim, 1))
        self._gaussians = torch.tensor(self._gaussians.astype(np.float32))
        self._weights = torch.tensor(weights)
        self._weights = torch.reshape(weights, shape=(self._dim, 1))
        self._strike = torch.tensor(strike)
    
    
    def BSNDPath(self, startTime, endTime, spots, vols, rfr, corrMat):
        
        spots = torch.tensor(spots.astype(np.float32))
        rfr = torch.tensor(rfr)
        corrMat = torch.tensor(corrMat)
        vols =  torch.tensor(vols)
        covMat = torch.diagflat(vols) @ corrMat @ torch.diagflat(vols)
        T = endTime- startTime
        T = torch.tensor(T)
        paths = torch.empty(size=(self._nbSimus, self._dim, 2, 3))
        paths[:, :, 0, 0] = spots # paths initialization to the initial spots
        
        chol = torch.empty(size=(self._dim, self._dim))
        
        if np.linalg.det(covMat)!=0:
            chol = np.linalg.cholesky(covMat)
            
        else:
            D, P = np.linalg.eig(covMat)
            self._chol = P @ np.sqrt(np.diag(D))
            
        chol = torch.tensor(chol.astype(np.float32))
        
        
        gaussianTerm = torch.sqrt(T) * torch.diagflat(spots) @ chol @ self._gaussians
        gaussianTerm = torch.reshape(gaussianTerm, shape=(self._nbSimus, self._dim))
        
        p00 =  paths[:, :, 0, 0].clone()
        paths[:, :, 1, 2] = p00* (1 + rfr * T)
        
        p12 = paths[:, :, 1, 2].clone()
        
        paths[:, :, 1, 0] = p12 + gaussianTerm
        paths[:, :, 1, 1] = p12 - gaussianTerm
        
        
        return paths
        
    
    
    def MCPrice(self, startTime, endTime, spots, vols, rfr, corrMat):
        
        
        start = time.time()
        
        T = endTime - startTime
        T = torch.tensor(T)
        dt = (endTime - startTime)/self._nbSteps
        dt = torch.tensor(dt)
        
        
        
        paths= self.BSNDPath(startTime, endTime, spots, vols, rfr, corrMat)
        
        #Reinitializing the values if we reuse the function for the same vibrato object
        self._payoffs = torch.empty(size=(nbSimus,1))
        
        lastSpots = torch.empty(size=(self._nbSimus, 1, 2)) # We have to generate the standard value, and the antithetic counterpart
        payoffs = torch.empty(size=(self._nbSimus,))
        payoffsAnti = torch.empty(size=(self._nbSimus,))
        
        lastSpots = paths[:, :, 1, :]
        payoffs = torch.maximum(lastSpots[:, :, 0] @ self._weights - self._strike, torch.tensor(0))
        payoffsAnti = torch.maximum(lastSpots[:, :, 1] @ self._weights - self._strike, torch.tensor(0))
        
        end=time.time()
        print(end-start)
        return 0.5 * torch.exp(-rfr * T) * (payoffs + payoffsAnti)
    
    
    
    def ComputeDelta(self, startTime, endTime, spots, vols, rfr, corrMat, eps):
        
        start=time.time()
        deltas = []
        for i in range(self._dim):
            
            a = self.MCPrice(startTime, endTime, spots + eps, vols, rfr, corrMat)
            b = self.MCPrice(startTime, endTime, spots - eps, vols, rfr, corrMat)
            delta = (a-b)/(2*eps)
            deltas.append( delta.numpy() )
        end=time.time()
        print(end-start)
        return deltas
    
    
    def ComputeGamma(self, startTime, endTime, spots, vols, rfr, corrMat, eps):
        
        start=time.time()
        gammas = []
        for i in range(self._dim):
            
            a = self.MCPrice(startTime, endTime, spots + eps, vols, rfr, corrMat)
            b = self.MCPrice(startTime, endTime, spots - eps, vols, rfr, corrMat)
            c = self.MCPrice(startTime, endTime, spots, vols, rfr, corrMat)
            gamma =  (a+b - 2*c)/(eps**2)
            gammas.append(gamma.numpy() )
        end=time.time()
        print(end-start)
        return gammas

### Tests

In [ ]:
dim=7
spots=np.array([100., 120., 90., 85., 145., 120., 102.])
rfr=0.05
corrMat = np.array([[1, 0, 0, 0, 0, 0, 0],
                    [0.9477, 1, 0, 0, 0, 0 ,0],
                    [0.8494, 0.7558, 1, 0, 0, 0, 0],
                    [0.8548, 0.7919, 0.982, 1, 0, 0, 0],
                    [0.8719, 0.8209, 0.9505, 0.9378, 1, 0, 0],
                    [0.6169, 0.6277, 0.6131, 0.64, 0.6417, 1, 0],
                    [0.7886, 0.7354, 0.9303, 0.8902, 0.8424, 0.5927, 1]])
corrMat = corrMat + corrMat.T - np.diag(np.diag(corrMat))
corrMat = torch.tensor(corrMat)
vols = np.array([0.146, 0.1925, 0.1712, 0.1679, 0.1688, 0.2192, 0.2068])
vols=torch.tensor(vols)
nbSimus=100000
weights = torch.tensor([1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7])
strike = 100
payoff = BasketCallPayoff(weights, strike)

startTime =0.
endTime = 1.

#### Par différences finies

Pour le prix: {'mean': tensor(15.6530),
 'var': tensor(12.6317),
 'lower': tensor(15.6310),
 'upper': tensor(15.6750)}, temps=0.05297517776489258
 
 Pour les delta, {'mean': tensor(0.8339),
 'var': tensor(0.0055),
 'lower': tensor(0.8335),
 'upper': tensor(0.8344)}, temps=0.7418444156646729
 
 Pour le Gamma, {'mean': tensor(297.7875),
 'var': tensor(1143706.6923),
 'lower': tensor(291.1592),
 'upper': tensor(304.4159)}, temps=1.1490263938903809
 
 Pour le gamma c'est très dépendant du eps. Les valeurs vont de valeurs négatives à des valeurs de l'ordres de plusieurs milliers selon le eps, illustrant la difficulté de la calibration des différences finies pour le calcul de sensitivités.

In [ ]:
eps = 0.00001

In [ ]:
fdND = FDND(dim, nbSimus, payoff, weights, strike)

In [ ]:
payoffs = fdND.MCPrice(startTime, endTime, spots, vols, rfr, corrMat)
monte_carlo(payoffs)

In [ ]:
deltas = fdND.ComputeDelta(startTime, endTime, spots, vols, rfr, corrMat, eps)

In [ ]:
deltas = np.array(deltas)
deltas = np.reshape(deltas, newshape=(nbSimus, dim))
dweightsNp = weights.numpy()
deltas = np.average(deltas, axis=1, weights=weightsNp) 
monte_carlo(torch.tensor(deltas))

In [ ]:
gammas = fdND.ComputeGamma(startTime, endTime, spots, vols, rfr, corrMat, eps)

In [ ]:
gammas = np.array(gammas)
gammas = np.reshape(gammas, newshape=(nbSimus, dim))
dweightsNp = weights.numpy()
gammas = np.average(gammas, axis=1, weights=weightsNp) 
monte_carlo(torch.tensor(gammas))

#### Par VAD

On a ici les gammas suivants pour chacun des spots:
    [ 0.0085,  0.0011, -0.0125,  0.0034,  0.0040,  0.0027,  0.0064], time=0.48844051361083984
    
Pour le delta:
{'mean': tensor(0.8092, grad_fn=<MeanBackward0>),
 'var': tensor(34.1091, grad_fn=<VarBackward0>),
 'lower': tensor(0.7730, grad_fn=<SubBackward0>),
 'upper': tensor(0.8454, grad_fn=<AddBackward0>)}, 
    
Pour le prix:
    {'mean': tensor(15.6482, grad_fn=<MeanBackward0>),
 'var': tensor(12.5145, grad_fn=<VarBackward0>),
 'lower': tensor(15.6263, grad_fn=<SubBackward0>),
 'upper': tensor(15.6702, grad_fn=<AddBackward0>)}

In [ ]:
bsND = BlackScholesND(dim, spots, rfr, corrMat, vols, nbSimus)

In [ ]:
bsND.Simulate(startTime, endTime)

In [ ]:
vibND = EuropeanVibratoND(payoff, bsND)

In [ ]:
vibND.ComputeVibrato(startTime, endTime, strike, weights)

In [ ]:
del out

In [ ]:
start = time.time()
out = vibND._deltaVibrato.mean()
out.backward()
gamma = -bsND._spots.grad
end = time.time()
print(end-start)

In [ ]:
gamma

In [ ]:
monte_carlo(-vibND._deltaVibrato)

In [ ]:
monte_carlo(vibND._payoffs)